In [86]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [73]:
STORM_NUMBER = 9
TITLE = "18 February 2022"
SUBTITLE = "8 AM 19 February 2022"
SETPOINT_LOCS = {}
sts = utc(2022, 2, 18, 18)
ets = utc(2022, 2, 19, 20, 0)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [87]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        geo = gpd.points_from_xy(x=[SETPOINT_LOCS[sp][0]], y=[SETPOINT_LOCS[sp][1]], crs=df.crs)
        df.loc[10000 + sp] = {"geo": geo[0], "val": val, "used_for_analysis": True,
                             "plotme": True}
    df["geo"] = gpd.GeoSeries(df["geo"], crs=df.crs)

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [100]:
def main():
    setpoints = [
        (23, 0),
        (24, 0),
        (9, 0),
        (8, 0),
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = []
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=6)
    # draw_setpoints(mp)

    plotsqw(mp)
    res = mp.postprocess(filename='220219.png')
    mp.close()

In [101]:
main()

<GeometryArray>
Length: 1, dtype: geometry
<GeometryArray>
Length: 1, dtype: geometry
<GeometryArray>
Length: 1, dtype: geometry
<GeometryArray>
Length: 1, dtype: geometry
0        POINT (-2374765.537 2745500.115)
1        POINT (-2121313.808 2724203.864)
2          POINT (1065975.126 266841.354)
3          POINT (891956.125 -440771.576)
4           POINT (872687.922 256081.060)
                       ...               
791        POINT (-909797.896 246411.060)
10023      POINT (594293.695 -420998.229)
10024      POINT (639341.209 -416962.807)
10009      POINT (643370.895 -462182.454)
10008      POINT (598038.831 -466235.958)
Name: geo, Length: 796, dtype: geometry


/opt/miniconda3/envs/prod/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/miniconda3/envs/prod/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/miniconda3/envs/prod/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/miniconda3/envs/prod/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array 

      state  wfo   val       lon      lat                               geo  \
0        AK  AFC  18.0 -149.9600  62.1800  POINT (-2374765.537 2745500.115)   
1        AK  AFG  16.0 -145.7500  63.4200  POINT (-2121313.808 2724203.864)   
2        MI  MQT  10.0  -85.9700  46.5600    POINT (1065975.126 266841.354)   
3        IL  ILX   9.0  -89.4200  40.5200    POINT (891956.125 -440771.576)   
4        MI  MQT   7.0  -88.5000  46.7400     POINT (872687.922 256081.060)   
...     ...  ...   ...       ...      ...                               ...   
791      MT  TFX   0.0 -111.9636  46.6056    POINT (-909797.896 246411.060)   
10023   NaN  NaN   0.0       NaN      NaN    POINT (594293.695 -420998.229)   
10024   NaN  NaN   0.0       NaN      NaN    POINT (639341.209 -416962.807)   
10009   NaN  NaN   0.0       NaN      NaN    POINT (643370.895 -462182.454)   
10008   NaN  NaN   0.0       NaN      NaN    POINT (598038.831 -466235.958)   

       used_for_analysis nwsli  plotme source  xcel

<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.